# Mengengerüst

Dieses Notebook widmet sich der Analyse einer XLS-Datei aus dem Warenwirtschaftssystem und der anschließenden Datenaufbereitung zur Erstellung eines Mengengerüsts. Ziel ist es, die Rohdaten zu bereinigen, in ein benutzerfreundliches Format zu transformieren, herstellerspezifische Informationen zu neutralisieren und die aufbereiteten Ergebnisse in ein Excel-Template zu exportieren.

Hieraus ergibt sich folgender Aufbau für dieses Notebook:
1. Laden der Daten aus dem Warenwirtschaftsystem
2. Extraktion der Daten
3. Neutralisierung
4. Übertragung der Daten in das Mengengerüst
5. Export

In [1]:
# Instalationen und Imports
#pip install spacy
#python -m spacy download de_core_news_sm
# pip install pandas xlrd
import pandas as pd
import json
import spacy
from spacy.training.example import Example
from transformers import pipeline

# Für Export in eine Excel Tabelle
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font, NumberFormatDescriptor
from openpyxl.utils.dataframe import dataframe_to_rows

c:\Users\sandra.nuissl\Desktop\Automatisierung_von_Ausschreibungen\venv_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Laden der Daten (Warenwirtschaftssystem)

Die Daten können mithilfe von Pandas problemlos aus der XLS-Datei in Python importiert werden. Obwohl eine einheitliche Formatierung vorliegt, enthält die Tabelle zahlreiche leere Spalten und Zeilen, die nicht benötigt werden und daher entfernt werden. Für eine höhere Automatisierung könnte das Warenwirtschaftssystem direkt über eine API mit der Codebase verbunden werden. Dadurch würde der manuelle Export und Import der Datei entfallen, wodurch Medienbrüche vermieden werden. Gleichzeitig würden die für die Excel-Datei erforderlichen "Bereinigungsschritte" überflüssig werden.

In [2]:
# Benennen des Dateipfades
datei_name = "Angebot.xls"

In [3]:
# Lesen der Excel aus dem Warenwirtschaftssystem
df_warenwirtschaft = pd.read_excel(datei_name, sheet_name='Sheet1')

# Löschen der leeren Zielen
df_warenwirtschaft = df_warenwirtschaft.dropna(how='all').reset_index(drop=True)

# Löschen der leeren Spalten
df_warenwirtschaft = df_warenwirtschaft.dropna(axis=1, how='all')

# Ausgabe
df_warenwirtschaft

,Unnamed: 1,Unnamed: 4,Unnamed: 5,Unnamed: 7,Unnamed: 13,Unnamed: 20,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 29
0,Mittelstand-Digital Zentrum Franken\nRaum RET ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-10-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angebot 65180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ihre Kundennummer:,NaN,NaN,177777,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ihr Fachberater:,NaN,NaN,Bauer Gunnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
164,Angebot gültig bis:,NaN,NaN,2024-11-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,Lieferadresse:,NaN,NaN,Mittelstand-Digital Zentrum Franken\nRaum RET ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,Montagebedingungen:,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,Zahlungskonditionen:,NaN,NaN,Vorauskasse,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Die Informationen sind über die gesammte Tabelle verteilt, jedoch werden nicht alle Spalten benötigt, da diese lediglich Summenwerte oder Zusatzinformationen enthalten. Daher sollen folgende Spalten nicht weiter berücksichtigt werden:

In [4]:
# Prüfen auf Werte
count_non_empty_in_4 = df_warenwirtschaft['Unnamed: 4'].count()
count_non_empty_in_7 = df_warenwirtschaft['Unnamed: 7'].count()
count_non_empty_in_13 = df_warenwirtschaft['Unnamed: 13'].count()
count_non_empty_in_24 = df_warenwirtschaft['Unnamed: 24'].count()
count_non_empty_in_25 = df_warenwirtschaft['Unnamed: 25'].count()
count_non_empty_in_26 = df_warenwirtschaft['Unnamed: 26'].count()
count_non_empty_in_29 = df_warenwirtschaft['Unnamed: 29'].count()

# Ausgabe
print(f"Anzahl der Werte in dieser Spalte: {count_non_empty_in_4} \n {df_warenwirtschaft['Unnamed: 4'][10]} \n")
print(f"Anzahl der Werte in dieser Spalte: {count_non_empty_in_7} \n {df_warenwirtschaft['Unnamed: 7'][5]} \n")
print(f"Anzahl der Werte in dieser Spalte: {count_non_empty_in_13} \n {df_warenwirtschaft['Unnamed: 13'][1]} \n")
print(f"Anzahl der Werte in dieser Spalte: {count_non_empty_in_24} \n {df_warenwirtschaft['Unnamed: 24'][2]} \n")
print(f"Anzahl der Werte in dieser Spalte: {count_non_empty_in_25} \n {df_warenwirtschaft['Unnamed: 25'][0]} \n")
print(f"Anzahl der Werte in dieser Spalte: {count_non_empty_in_26} \n {df_warenwirtschaft['Unnamed: 26'][1]} \n")
print(f"Anzahl der Werte in dieser Spalte: {count_non_empty_in_29} \n {df_warenwirtschaft['Unnamed: 29'][0]} \n")

Anzahl der Werte in dieser Spalte: 2 
 nan 

Anzahl der Werte in dieser Spalte: 9 
 0911-95211-0 

Anzahl der Werte in dieser Spalte: 3 
 nan 

Anzahl der Werte in dieser Spalte: 2 
 nan 

Anzahl der Werte in dieser Spalte: 1 
 nan 

Anzahl der Werte in dieser Spalte: 1 
 nan 

Anzahl der Werte in dieser Spalte: 3 
 nan 



## 2. Extraktion der Daten

Nachdem identifiziert wurde in welchen Spalten sich die benötigten Informationen befinden, können diese in einem neuen Data Frame gespeichert und entsprechen benannt werden:
- Spalte 1 (Anzahl der Möbelstücke)
- Spalte 5 (Beschreibung und Positionsnummer)
- Spalte 20 (E-Preis/EUR)
- Spalte 27 (G-Preis/EUR)

Im nächsten Schritt sollen die relevanten Informationen iterativ aus den Spalten extrahiert, den jeweiligen Möbelstücken zugeordnet und in einem Dictionary gespeichert werden. Dies ist notwendig, da die Beschreibungstexte über mehrere Zeilen verteilt sind, jedoch in einer einzigen Variablen zusammengefasst werden müssen. Die Zuordnung erfolgt anhand der Positionsnummern: Alle Zeilen, die zwischen zwei Positionsnummern liegen, werden zu einem Artikel zusammengefasst.

Das Ergebnis ist eine Liste, die alle Artikel mit den zugehörigen Positionsnummern und den beschreibenden Informationen umfasst, darunter Beschreibungstext, Preise, Artikelnummer und benötigte Menge.

In [5]:
# Erstellung eines neuen Data Frames, welcher nur die benötigten Informationen enthält + Umbenennung
df_selected = df_warenwirtschaft[['Unnamed: 1', 'Unnamed: 5', 'Unnamed: 20', 'Unnamed: 27']].copy()
df_selected = df_selected.rename(columns={'Unnamed: 1':'menge',
                                          'Unnamed: 5':'beschreibung', 
                                          'Unnamed: 20':'e_preis', 
                                          'Unnamed: 27':'g_preis'})

# Ausgabe
df_selected.head(5)

,menge,beschreibung,e_preis,g_preis
0,Mittelstand-Digital Zentrum Franken\nRaum RET ...,NaN,NaN,NaN
1,2024-10-23 00:00:00,NaN,NaN,NaN
2,Angebot 65180,NaN,NaN,NaN
3,Ihre Kundennummer:,NaN,NaN,NaN
4,Ihr Fachberater:,NaN,NaN,NaN


In [6]:
# Liste zum Speichern der extrahierten Blöcke als Dictionaries
blocks = []

# Temporäre Variablen für jeden Block
current_pos = None
current_menge = None
current_e_preis = None
current_g_preis = None
current_artikelnummer = None
current_block = []

# Durch die Zeilen des DataFrames iterieren
for idx, row in df_selected.iterrows():
    beschreibung = row['beschreibung']
    menge = row['menge']
    e_preis = row['e_preis']
    g_preis = row['g_preis']
    
    # Überprüfen, ob eine neue Position ("Pos") gefunden wurde
    if beschreibung and "Pos" in str(beschreibung):
        # Wenn es einen aktiven Block gibt, als Dictionary speichern
        if current_block:
            blocks.append({
                'position': current_pos,
                'artikelnummer': current_artikelnummer,
                'beschreibung': current_block,
                'menge': current_menge,
                'e_preis': current_e_preis,
                'g_preis': current_g_preis
            })
        
        # Neue Position setzen
        current_pos = beschreibung
        current_block = []
        current_menge = None
        current_e_preis = None
        current_g_preis = None

        # Nächste Zeile als Artikelnummer speichern
        if idx + 1 < len(df_selected):
            next_row = df_selected.iloc[idx + 1]
            current_artikelnummer = next_row['beschreibung']  # Setze die Artikelnummer hier
            continue

    # Beschreibung zum Block hinzufügen, falls vorhanden
    if beschreibung and not "Pos" in str(beschreibung):
        # Füge nur die Beschreibung hinzu, wenn es keine Artikelnummer ist
        if current_artikelnummer and beschreibung != current_artikelnummer:
            current_block.append(beschreibung)

    # Menge, Einzelpreis und Gesamtpreis für den Block aktualisieren, wenn vorhanden
    if pd.notna(menge):
        current_menge = menge
    if pd.notna(e_preis):
        current_e_preis = e_preis
    if pd.notna(g_preis):
        current_g_preis = g_preis

# Letzten Block speichern, wenn er existiert
if current_block:
    blocks.append({
        'position': current_pos,
        'artikelnummer': current_artikelnummer,
        'beschreibung': current_block,
        'menge': current_menge,
        'e_preis': current_e_preis,
        'g_preis': current_g_preis
    })

# Konvertieren der Formatierungen
for block in blocks:
    # Sting zu Floats
    for key in ['e_preis', 'g_preis']:
        if block[key]:
            # Tausenderpunkte entfernen und Komma in Punkt umwandeln
            block[key] = float(block[key].replace('.', '').replace(',', '.'))
        else:
            block[key] = 0.0

    # Integers zu Strings
    for key in ['beschreibung']:
        block[key] = [str(item) for item in block[key]]

    # Beschreibung zu einem String umwandeln
    block['beschreibung'] = " ".join(block['beschreibung'])

# Ausgabe des dritten Blocks
blocks[0]

{'position': 'Pos. 1',
 'artikelnummer': 'MSE1T210S',
 'beschreibung': 'Migration SE Tisch mit Schiebeplatte\nTisch Lift Elektrisch H650-1250 T-Fuß,\nTyp 10/gerade Option: Größe: Standard\nBreite Standard: 1800\nTiefe Standard: 800\nÜberstand: ohne Überstand\nTyp Platte: 25mm Melamin Std\nFarbe Tischplatte: Snow WY\nFarbe Plattenkante: Schwarz 340/N0\nUntere Säule und Fußausleger: Pearl Schwarz PB\nInnere Säule(n): Pearl Schwarz PB\nSchiebeplatte: Facility-Manager SP1\nNetzkabel für Lifttisch: mit Kabel 3m DE/FR/CH\nBedienpaneel Lifttisch: Auf/Ab Bedienpaneel SC1\nElektrifizierung: Nein\nPlattenausfräsung: ohne\nKabelkanal: Standard K00',
 'menge': '2',
 'e_preis': 658.89,
 'g_preis': 1317.78}

## 3. Neutralisierung

In [7]:
# Beispieltext zur Veranschaulichung
text = "Der Tisch ist in Snow WY und Pearl Schwarz gehalten, und der Schreibtisch hat eine Migration SE Platte. Migration SE Kabelmanagement in Amsel MG mit Unterschrank und Kabinett links"


In [8]:
# Laden das deutschen Modells
nlp = spacy.load("de_core_news_lg")

# Laden das NER-Modells von Hugging Face
ner_pipeline = pipeline("ner", model="xlm-roberta-large-finetuned-conll03-german")

# Laden der Mapping-Tabelle
with open("mapping.json", "r", encoding="utf-8") as file:
    mapping = json.load(file)

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-german were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [9]:
# Erstellen einer Entity Ruler Pipeline
ruler = nlp.add_pipe("entity_ruler", before="ner")

# Erstellen der Muster für die Erkennung der Begriffe aus der Mapping-Tabelle
patterns = [{"label": "NEUTRAL", "pattern": key} for key in mapping.keys()]
ruler.add_patterns(patterns)

Es wurden 2 verschiedene Algorythmen zur Named Entity Recognition getestet:

In [10]:
# Verarbeitung des Textes mit SpaCy
doc = nlp(text)

# Ausgabe
for ent in doc.ents:
        print(f"{ent} \t wurde erkannt als {ent.label_}")

Snow WY 	 wurde erkannt als NEUTRAL
Pearl Schwarz 	 wurde erkannt als ORG
Migration SE 	 wurde erkannt als NEUTRAL
Migration SE 	 wurde erkannt als NEUTRAL
Amsel MG 	 wurde erkannt als MISC


In [11]:
# Führe NER auf dem Text aus
ner_results = ner_pipeline(text)

# Ausgabe
ner_results

[{'entity': 'I-LOC',
  'score': np.float32(0.39882267),
  'index': 5,
  'word': '▁Snow',
  'start': 17,
  'end': 21},
 {'entity': 'I-MISC',
  'score': np.float32(0.9966653),
  'index': 20,
  'word': '▁Migration',
  'start': 83,
  'end': 92},
 {'entity': 'I-MISC',
  'score': np.float32(0.9980629),
  'index': 21,
  'word': '▁SE',
  'start': 93,
  'end': 95},
 {'entity': 'I-MISC',
  'score': np.float32(0.94833714),
  'index': 25,
  'word': '▁Migration',
  'start': 104,
  'end': 113},
 {'entity': 'I-MISC',
  'score': np.float32(0.9334479),
  'index': 26,
  'word': '▁SE',
  'start': 114,
  'end': 116}]

Die erkannten Wörter werden darauf hin Regelbasiert ersetzt. Gibt es kein Wort, welches in der Mapping Tabelle hinterlegt ist, so wird mithilfe einer Similarity Search eine ähnliches Wort gesucht.

In [12]:
# Funktion zur Ähnlichkeitsanalyse für unbekannte Begriffe
def find_similar_term(term, known_terms, nlp):
    term_doc = nlp(term)
    best_match = None
    best_similarity = 0.0

    for known in known_terms:
        known_doc = nlp(known)
        similarity = term_doc.similarity(known_doc)
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = known

    return best_match if best_similarity > 0.7 else None

In [13]:
for ent in doc.ents:
    # Prüfen, ob ein Begriff in der Mapping Tabelle vorhanden ist
    if ent.label_ == "NEUTRAL" and ent.text in mapping:
        neutral_description = text.replace(ent.text, mapping[ent.text])
        print(f"{ent.text} \t wurde ersetzt durch {mapping[ent.text]}")
    else:
        # Ähnlichkeitsprüfung falls kein Word vorhanden
        similar_term = find_similar_term(ent.text, mapping.keys(), nlp)
        if similar_term:
            neutral_description = text.replace(ent.text, mapping[similar_term])
            print(f"{ent.text} \t wurde ersetzt durch {mapping[similar_term]}")
        else:
            # Als zu bearbeitend markieren, falls nichts gefunden
            neutral_description = text.replace(ent.text, "zu bearbeiten")
            print(f"{ent.text} \t konnte nicht ersetzt werden, wurde durch 'zu bearbeiten' ersetzt.")

Snow WY 	 wurde ersetzt durch Weiß
Pearl Schwarz 	 konnte nicht ersetzt werden, wurde durch 'zu bearbeiten' ersetzt.
Migration SE 	 wurde ersetzt durch 
Migration SE 	 wurde ersetzt durch 
Amsel MG 	 konnte nicht ersetzt werden, wurde durch 'zu bearbeiten' ersetzt.


Übertragen auf den Datensatz:

In [14]:
# Funktion zur Neutralisierung des Textes mit Mapping und Ähnlichkeitsprüfung
def neutralize_entities_in_text(text, mapping, nlp):
    doc = nlp(text)
    neutral_description = text
    ersetzungen = {}

    for ent in doc.ents:
        original_word = ent.text
        replaced_word = ""

        # Prüfen, ob ein Begriff in der Mapping-Tabelle vorhanden ist
        if ent.label_ == "NEUTRAL" and original_word in mapping:
            replaced_word = mapping[original_word]
        else:
            # Ähnlichkeitsprüfung, falls kein passender Begriff in der Mapping-Tabelle vorhanden ist
            similar_term = find_similar_term(original_word, mapping.keys(), nlp)
            if similar_term:
                replaced_word = mapping[similar_term]
            else:
                # Falls auch keine Ähnlichkeit gefunden wird, als 'zu bearbeiten' markieren
                replaced_word = "zu bearbeiten"

        # Ersetze das Wort im Text
        neutral_description = neutral_description.replace(original_word, replaced_word)

        # Speichere die Ersetzung im Dictionary
        ersetzungen[original_word] = replaced_word

    return neutral_description, ersetzungen


In [15]:
# Durchlaufe die Liste von Dictionaries
for block in blocks:
    # Neue Keys hinzufügen
    block["neutralisierung"] = []
    block["neutralisierte_beschreibung"] = ""
    block["ersetzungen"] = {}

    # Anwenden der NER und Wörter ermitteln
    description = block["beschreibung"]

    # Neutralisierung durchführen und Ersetzungen speichern
    neutral_description, ersetzungen = neutralize_entities_in_text(description, mapping, nlp)

    # Füge die modifizierten Daten dem Dictionary hinzu
    block["neutralisierung"].extend(ersetzungen.keys())
    block["neutralisierte_beschreibung"] = neutral_description
    block["ersetzungen"] = ersetzungen

# Ausgabe der modifizierten Liste
blocks


C:\Users\sandra.nuissl\AppData\Local\Temp\ipykernel_12932\970985569.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = term_doc.similarity(known_doc)
C:\Users\sandra.nuissl\AppData\Local\Temp\ipykernel_12932\970985569.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = term_doc.similarity(known_doc)
C:\Users\sandra.nuissl\AppData\Local\Temp\ipykernel_12932\970985569.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = term_doc.similarity(known_doc)
C:\Users\sandra.nuissl\AppData\Local\Temp\ipykernel_12932\970985569.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = term_doc.similarity(known_doc)
C:\Users\sandra.nuissl\AppData\Local\Temp\ipykernel_12932\970985569.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = term_doc.similarity(known_doc)
C:\Users\sandra.nuissl\AppData\Local\Temp\ipykerne

[{'position': 'Pos. 1',
  'artikelnummer': 'MSE1T210S',
  'beschreibung': 'Migration SE Tisch mit Schiebeplatte\nTisch Lift Elektrisch H650-1250 T-Fuß,\nTyp 10/gerade Option: Größe: Standard\nBreite Standard: 1800\nTiefe Standard: 800\nÜberstand: ohne Überstand\nTyp Platte: 25mm Melamin Std\nFarbe Tischplatte: Snow WY\nFarbe Plattenkante: Schwarz 340/N0\nUntere Säule und Fußausleger: Pearl Schwarz PB\nInnere Säule(n): Pearl Schwarz PB\nSchiebeplatte: Facility-Manager SP1\nNetzkabel für Lifttisch: mit Kabel 3m DE/FR/CH\nBedienpaneel Lifttisch: Auf/Ab Bedienpaneel SC1\nElektrifizierung: Nein\nPlattenausfräsung: ohne\nKabelkanal: Standard K00',
  'menge': '2',
  'e_preis': 658.89,
  'g_preis': 1317.78,
  'neutralisierung': ['Migration SE',
   'H650',
   'T-Fuß',
   'Snow WY',
   'N0',
   'Pearl Schwarz PB\nInnere Säule',
   'Facility-Manager SP1',
   'Lifttisch',
   'DE',
   'CH',
   'Bedienpaneel',
   'Standard K00'],
  'neutralisierte_beschreibung': ' Tisch mit Schiebeplatte\nTisch Lift

## 4. Übertragung der Daten in das Mengengerüst

Nach Abschluss aller Bereinigungen und der Neutralisierung können die aufbereiteten Daten in die finale Struktur des DataFrames übertragen werden. Dabei ist zu berücksichtigen, dass zwei unterschiedliche Download-Optionen bereitgestellt werden sollen. Entsprechend müssen die Spalten je nach Option unterschiedlich strukturiert werden, und gegebenenfalls sind die Inhalte anzupassen.

1. Kostenschätzung (intern)
2. Mengengerüst (extern)

### Data Frame für interne Kostenschätzung

Die interne Kostenschätzung dient dem Unternehmen als Übersicht und Orientierung. Dabei bleiben die Preise erhalten, um eine Vorstellung vom finanziellen Rahmen zu ermöglichen. Zudem ist es nicht erforderlich, die Beschreibungstexte zu neutralisieren. Auch die Artikelnummern sollen in der Tabelle unverändert beibehalten werden.

In [16]:
# Liste zur Speicherung der neuen Zeilen
new_rows_intern = []

# Iteration über die Daten und hinzufügt der Werte zur Liste
for entry in blocks:
    if entry['position'] is not None:
        # Werte aufbereiten
        pos = entry['position'].replace('Pos. ', '')
        artikelnummer = entry['artikelnummer']
        beschreibung = "\n".join(entry['beschreibung']) + "\n" if isinstance(entry['beschreibung'], list) else entry['beschreibung'] + "\n"
        menge = entry['menge']
        e_preis = entry['e_preis']
        g_preis = entry['g_preis']
        
        # Neue Zeile zur Liste hinzufügen
        new_rows_intern.append({
            'Pos': pos,
            'Hersteller': None,
            'Bezeichnung': beschreibung,
            'Artikelnummer': artikelnummer,
            'Menge': menge,
            'Einzelpreis / EUR netto': e_preis,
            'Gesamtpreis / EUR netto': g_preis
        })

# DataFrame aus der Liste erstellen
df_intern = pd.DataFrame(new_rows_intern)

# DataFrame ausgeben
df_intern.head(5)

,Pos,Hersteller,Bezeichnung,Artikelnummer,Menge,Einzelpreis / EUR netto,Gesamtpreis / EUR netto
0,1,None,Migration SE Tisch mit Schiebeplatte\nTisch Li...,MSE1T210S,2,658.89,1317.78
1,1.1,None,Migration SE Kabelmanagement Tisch\nVertikal K...,MSEA9000,2,41.50,83.00
2,2.1,None,Migration SE Tisch ohne Schiebeplatte\nTisch L...,MSE1T210,4,658.89,2635.56
3,2.1.1,None,Migration SE Kabelmanagement Tisch\nVertikal K...,MSEA9000,4,41.50,166.00
4,2.2,None,Netline Nak \nNetzanfangskabel D / A / F / BE ...,AHM,4,8.07,32.28


### Data Frame für externes Mengengerüst

Das Mengengerüst wird für externe Interessensgruppen im Rahmen der Ausschreibung erstellt. Es dient dazu, den Bewerbern die Anforderungen an die Möbelstücke sowie die gewünschte Menge klar und prägnant zu vermitteln. Preise und Artikelnummern müssen aus diesem DataFrame entfernt werden, da die Unternehmen ihre eigenen Angebotspreise eintragen sollen. Darüber hinaus müssen die Beschreibungen herstellerneutral formuliert sein, um eine faire Vergleichbarkeit der Angebote zu gewährleisten.

In [17]:
# Liste zur Speicherung der neuen Zeilen
new_rows_extern = []

# Iteration über die Daten und hinzufügt der Werte zur Liste
for entry in blocks:
    if entry['position'] is not None:
        # Werte aufbereiten
        pos = entry['position'].replace('Pos. ', '')
        beschreibung = "\n".join(entry['neutralisierte_beschreibung']) + "\n" if isinstance(entry['neutralisierte_beschreibung'], list) else entry['neutralisierte_beschreibung'] + "\n"
        menge = entry['menge']
        
        # Neue Zeile zur Liste hinzufügen
        new_rows_extern.append({
            'Pos': pos,
            'Bezeichnung': beschreibung,
            'Menge': menge,
            'Einzelpreis / EUR netto': None,
            'Gesamtpreis / EUR netto': None
        })

# DataFrame aus der Liste erstellen
df_extern = pd.DataFrame(new_rows_extern)

# DataFrame ausgeben
df_extern.head(5)

,Pos,Bezeichnung,Menge,Einzelpreis / EUR netto,Gesamtpreis / EUR netto
0,1,Tisch mit Schiebeplatte\nTisch Lift Elektrisc...,2,None,None
1,1.1,Kabelmanagement Tisch\nVertikal Kabelkanal fü...,2,None,None
2,2.1,Tisch ohne Schiebeplatte\nTisch Lift Elektris...,4,None,None
3,2.1.1,Kabelmanagement Tisch\nVertikal Kabelkanal fü...,4,None,None
4,2.2,zu bearbeiten \nzu bearbeiten / zu bearbeiten ...,4,None,None


## 5. Export

Im letzten Schritt wird die erstellte Datei im gewünschten Excel-Format heruntergeladen. Dafür wird zunächst ein Template erstellt, in das anschließend die Informationen aus dem DataFrame eingetragen werden. Die fertige Datei wird schließlich im XLSX-Format gespeichert.
Dabei gibt es kleinere Unterschiede zwischen der internen Datei für die Kostenschätzung und dem externen Mengengerüst. Diese betreffen die bereitgestellten Informationen sowie die Bearbeitungsmöglichkeiten, die je nach Verwendungszweck entsprechend angepasst werden.

### Intern

Es werden hier die Daten aus dem Data Frame zu der internen Kostenschätzung übernommen.

In [18]:
# Pfad zur Excel-Datei (Speichername)
excel_path = 'export_intern.xlsx'

# Erstellen eines neuen Excel-Dokuments
wb = Workbook()
ws = wb.active

# Hinzufügen benutzerdefinierter Zeilen (Später vielleicht mit Frontend und Template verbinden)
ws.append(['Muster Uni'])
ws.append([])
ws.append(['Projekt: Musterstadt, Musterstraße 7'])
ws.append(['Los 1 - Büromobiliar'])
ws.append([])

# Border-Definition für dünne Linien
thin_border = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))

# Schriftart für die gesamte Tabelle
header_font = Font(size=10, color="006100")
default_font = Font(size=10)

# Alignment-Definition für vertikale Ausrichtung
vertical_top_alignment = Alignment(vertical='top', wrap_text=True)

# DataFrame in die Excel-Datei schreiben (beginnend ab der 6. Zeile)
for r_idx, r in enumerate(dataframe_to_rows(df_intern, index=False, header=True), 1):
    ws.append(r)

    # Formatierung
    for cell in ws[r_idx + 5]:
        cell.border = thin_border
        cell.font = default_font
        cell.alignment = vertical_top_alignment

    # Wenn es die erste Datenzeile ist (Header), dann formatieren wir diese Zeile
    if r_idx == 1:
        for cell in ws[r_idx + 5]:
            cell.border = thin_border
            cell.fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')
            cell.font = header_font
            cell.alignment = vertical_top_alignment

    # Formatierungen für spezifische Spalten
    if r_idx + 5 > 5:
        ws.cell(row=r_idx + 5, column=1).number_format = '@'     # Pos als Text
        ws.cell(row=r_idx + 5, column=5).number_format = '0'     # Menge als Zahl ohne Nachkommastellen
        ws.cell(row=r_idx + 5, column=6).number_format = '0.00'  # Einzelpreis als Zahl mit 2 Nachkommastellen

        # Formel für den Gesamtpreis nur ab der 2. Datenzeile (r_idx > 1)
        if r_idx > 1:
            ws.cell(row=r_idx + 5, column=7).value = f"=E{r_idx + 5}*F{r_idx + 5}"
            ws.cell(row=r_idx + 5, column=7).number_format = '0.00'

        # Textumbruch für die Beschreibung aktivieren
        description_cell = ws.cell(row=r_idx + 5, column=3)
        description_cell.alignment = vertical_top_alignment
        description_cell.font = default_font

        # Füllung für die Einzelpreis-Spalte
        single_price_cell = ws.cell(row=r_idx + 5, column=6)
        if r_idx > 1:
            single_price_cell.fill = PatternFill(start_color='D9D9D9', end_color='D9D9D9', fill_type='solid')

# Benutzerdefinierte Zeilen
for row in range(1, 5):
    for cell in ws[row]:
        cell.font = Font(name="Calibri", size=14)

# Spaltenbreiten festlegen
ws.column_dimensions['A'].width = 10  # Pos
ws.column_dimensions['B'].width = 20  # Hersteller
ws.column_dimensions['C'].width = 60  # Bezeichnung
ws.column_dimensions['D'].width = 25  # Artikelnummer
ws.column_dimensions['E'].width = 20  # Menge
ws.column_dimensions['F'].width = 30  # Einzelpreis
ws.column_dimensions['G'].width = 30  # Gesamtpreis

# Zoomstufe einstellen
ws.sheet_view.zoomScale = 80

# Gitternetzlinien ausblenden
ws.sheet_view.showGridLines = False

# Excel-Datei speichern
wb.save(excel_path)


### Extern

Für dieses File werden die Daten aus dem Data Frame zum externen Mengengerüst übernommen.
Außerdem unterscheidet sich die Formatierung etwas.

In [19]:
# Pfad zur Excel-Datei (Speichername)
excel_path = 'export_extern.xlsx'

# Erstellen eines neuen Excel-Dokuments
wb = Workbook()
ws = wb.active

# Hinzufügen benutzerdefinierter Zeilen (Später vielleicht mit Frontend und Template verbinden)
ws.append(['Muster Uni'])
ws.append([])
ws.append(['Projekt: Musterstadt, Musterstraße 7'])
ws.append(['Los 1 - Büromobiliar'])
ws.append([])

# Border-Definition für dünne Linien
thin_border = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))

# Schriftart für die gesamte Tabelle
header_font = Font(size=10, color="006100")
default_font = Font(size=10)

# Alignment-Definition für vertikale Ausrichtung
vertical_top_alignment = Alignment(vertical='top', wrap_text=True)

# DataFrame in die Excel-Datei schreiben (beginnend ab der 6. Zeile)
for r_idx, r in enumerate(dataframe_to_rows(df_extern, index=False, header=True), 1):
    ws.append(r)

    # Formatierung
    for cell in ws[r_idx + 5]:
        cell.border = thin_border
        cell.font = default_font
        cell.alignment = vertical_top_alignment

    # Wenn es die erste Datenzeile ist (Header), dann formatieren wir diese Zeile
    if r_idx == 1:
        for cell in ws[r_idx + 5]:
            cell.border = thin_border
            cell.fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')
            cell.font = header_font
            cell.alignment = vertical_top_alignment

    # Formatierungen für spezifische Spalten
    if r_idx + 5 > 5:
        ws.cell(row=r_idx + 5, column=1).number_format = '@'     # Pos als Text
        ws.cell(row=r_idx + 5, column=3).number_format = '0'     # Menge als Zahl ohne Nachkommastellen
        ws.cell(row=r_idx + 5, column=4).number_format = '0.00'  # Einzelpreis als Zahl mit 2 Nachkommastellen

        # Formel für den Gesamtpreis nur ab der 2. Datenzeile (r_idx > 1)
        if r_idx > 1:
            ws.cell(row=r_idx + 5, column=5).value = f"=C{r_idx + 5}*D{r_idx + 5}"
            ws.cell(row=r_idx + 5, column=5).number_format = '0.00'

        # Textumbruch für die Beschreibung aktivieren
        description_cell = ws.cell(row=r_idx + 5, column=2)
        description_cell.alignment = vertical_top_alignment
        description_cell.font = default_font

        # Füllung für die Einzelpreis-Spalte
        single_price_cell = ws.cell(row=r_idx + 5, column=4)
        if r_idx > 1:
            single_price_cell.fill = PatternFill(start_color='D9D9D9', end_color='D9D9D9', fill_type='solid')

# Benutzerdefinierte Zeilen
for row in range(1, 5):
    for cell in ws[row]:
        cell.font = Font(name="Calibri", size=14)

# Spaltenbreiten festlegen
ws.column_dimensions['A'].width = 10  # Pos
ws.column_dimensions['B'].width = 60  # Bezeichnung
ws.column_dimensions['C'].width = 20  # Menge
ws.column_dimensions['D'].width = 30  # Einzelpreis
ws.column_dimensions['E'].width = 30  # Gesamtpreis

# Zoomstufe einstellen
ws.sheet_view.zoomScale = 80

# Gitternetzlinien ausblenden
ws.sheet_view.showGridLines = False

# Excel-Datei speichern
wb.save(excel_path)